## CellChat analysis of multiple spatial transcriptomics datasets
Suoqin Jin
22 February, 2024
Load the required libraries
Part I: Data input & processing and initialization of CellChat object
Load data
Create a CellChat object
Set the ligand-receptor interaction database
Preprocessing the expression data for cell-cell communication analysis
Part II: Inference of cell-cell communication network
Compute the communication probability and infer cellular communication network
Infer the cell-cell communication at a signaling pathway level
Calculate the aggregated cell-cell communication network
Part III: Visualization of cell-cell communication network
Compute the contribution of each ligand-receptor pair to the overall signaling pathway
Part V: Save the CellChat object
This vignette outlines the steps of inference, analysis and visualization of cell-cell communication network for multiple spatial transcriptomics datasets using CellChat. We showcase CellChat’s application to multiple spatial transcriptomics datasets by applying it to two replicates from human spatial intestine datasets, which were downloaded from https://simmonslab.shinyapps.io/FetalAtlasDataPortal/.

Below we briefly describe the key steps of applying CellChat to multiple spatial transcriptomics datasets. Please check the vignette of applying CellChat to an individual spatially resolved transcriptomics dataset for detailed descriptions of the methods and steps, and check the vignette of FAQ on applying CellChat to spatially resolved transcriptomics data for detailed descriptions of applying CellChat to different types of spatial transcriptomics data.

Load the required libraries

In [ ]:
rm(list = ls())
library(CellChat)
library(patchwork)
library(Seurat)

In [ ]:
setwd("/media/bio/Disk/Research Data/EBV/omicverse")

#### load the CellChat object

In [ ]:
cellchat <- readRDS(file = "Processed Data/GSE206245_NPC_ST_Cluster_Tangram_Cellchat.rds")

Subset CellChat object using a portion of cells

In [ ]:
cellchat_Niche4 <- readRDS(file = "Processed Data/GSE206245_NPC_ST_Cluster_Tangram_Cellchat_Niche4.rds")
cellchat_Niche4

cellchat_NonNiche4 <- readRDS(file = "Processed Data/GSE206245_NPC_ST_Cluster_Tangram_Cellchat_NonNiche4.rds")
cellchat_NonNiche4

In [ ]:
object.list <- list(Other.Niches = cellchat_NonNiche4, Niche4 = cellchat_Niche4)
cellchat <- mergeCellChat(object.list, add.names = names(object.list))
cellchat

In [ ]:
compareInteractions(cellchat, show.legend = F, group = c(1,2), color.use = c("darkred","skyblue"), measure = "weight")

pdf("Results/10.NPC_ST_Analysis/Spatial_compareInteractions.pdf", width = 4, height = 5)  # You can adjust width and height as needed
compareInteractions(cellchat, show.legend = F, group = c(1,2), color.use = c("darkred","skyblue"), measure = "weight")
dev.off()

In [ ]:
netVisual_diffInteraction(cellchat, sources.use = c("Tumor","SPP1..Macro", "IL1B..Macro", "C1QC..Macro"), comparison = c(1,2),
targets.use = c("Tumor","SPP1..Macro", "IL1B..Macro", "C1QC..Macro"), weight.scale = T, measure = "weight")

pdf("Results/10.NPC_ST_Analysis/Spatial_netVisual_diffInteraction.pdf", width = 6, height = 6)  # You can adjust width and height as needed
netVisual_diffInteraction(cellchat, sources.use = c("Tumor","SPP1..Macro", "IL1B..Macro", "C1QC..Macro"), comparison = c(1,2),
targets.use = c("Tumor","SPP1..Macro", "IL1B..Macro", "C1QC..Macro"), weight.scale = T, measure = "weight")
dev.off()
dev.off()

In [ ]:
netVisual_heatmap(cellchat, sources.use = c("Tumor","SPP1..Macro", "IL1B..Macro", "C1QC..Macro"), comparison = c(1,2),
targets.use = c("Tumor","SPP1..Macro", "IL1B..Macro", "C1QC..Macro"), measure = "weight", color.heatmap = c('#2166ac','#b2182b'))

pdf("Results/10.NPC_ST_Analysis/Spatial_netVisual_heatmap.pdf", width = 5, height = 4.5)  # You can adjust width and height as needed
netVisual_heatmap(cellchat, sources.use = c("Tumor","SPP1..Macro", "IL1B..Macro", "C1QC..Macro"), comparison = c(1,2),
targets.use = c("Tumor","SPP1..Macro", "IL1B..Macro", "C1QC..Macro"), measure = "weight", color.heatmap = c('#2166ac','#b2182b'))
dev.off()

In [ ]:
pdf("Results/10.NPC_ST_Analysis/Spatial_netVisual_aggregate_SPP1_MIF.pdf.pdf", width = 10, height = 8)  # You can adjust width and height as needed

par(mfrow = c(2,2), xpd=TRUE)

pathways.show <- c("MIF") 
weight.max <- getMaxWeight(object.list, slot.name = c("netP"), attribute = pathways.show) # control the edge weights across different datasets
for (i in 1:length(object.list)) {
  netVisual_aggregate(object.list[[i]], signaling = pathways.show, layout = "circle", edge.weight.max = weight.max[1], edge.width.max = 10, 
  sources.use = c("Tumor"),
targets.use = c("Tumor","SPP1..Macro", "IL1B..Macro", "C1QC..Macro"),
signaling.name = paste(pathways.show, names(object.list)[i]))
}

pathways.show <- c("SPP1") 
weight.max <- getMaxWeight(object.list, slot.name = c("netP"), attribute = pathways.show) # control the edge weights across different datasets
for (i in 1:length(object.list)) {
  netVisual_aggregate(object.list[[i]], signaling = pathways.show, layout = "circle", edge.weight.max = weight.max[1], edge.width.max = 10, 
  sources.use = c("SPP1..Macro"),
targets.use = c("Tumor","SPP1..Macro", "IL1B..Macro", "C1QC..Macro"),
signaling.name = paste(pathways.show, names(object.list)[i]))
}

dev.off()

In [ ]:
rankNet(cellchat, mode = "comparison", measure = "weight", color.use = c("darkred","skyblue"), signaling = c("CCL","CXCL","FGF","SPP1","MIF"), stacked = T, do.stat = TRUE)

pdf("Results/10.NPC_ST_Analysis/Spatial_rankNet.pdf", width = 4, height = 6)  # You can adjust width and height as needed
rankNet(cellchat, mode = "comparison", measure = "weight", signaling = c("CCL","CXCL","FGF","SPP1","MIF"), stacked = T, do.stat = TRUE)
dev.off()

In [ ]:
pairLR.use <- extractEnrichedLR(cellchat, signaling = c("CCL","CXCL","EGF","SPP1"))
n1 <- netVisual_bubble(cellchat, sources.use = c("SPP1..Macro"), targets.use = c("Tumor","SPP1..Macro", "IL1B..Macro", "C1QC..Macro"), pairLR.use = pairLR.use, comparison = c(1, 2), angle.x = 45)
n1


pdf("Results/10.NPC_ST_Analysis/Spatial_netVisual_bubble1.pdf", width = 8, height = 6)  # You can adjust width and height as needed
netVisual_bubble(cellchat, sources.use = c("SPP1..Macro"), targets.use = c("Tumor","SPP1..Macro", "IL1B..Macro", "C1QC..Macro"), pairLR.use = pairLR.use, comparison = c(1, 2), angle.x = 45)
n1
dev.off()


In [ ]:
pairLR.use <- extractEnrichedLR(cellchat, signaling = c("CCL","CXCL","EGF","MIF"))
n2 <- netVisual_bubble(cellchat, sources.use = c("Tumor"), targets.use = c("Tumor","SPP1..Macro", "IL1B..Macro", "C1QC..Macro"), pairLR.use = pairLR.use, comparison = c(1, 2), angle.x = 45)
n2

pdf("Results/10.NPC_ST_Analysis/Spatial_netVisual_bubble2.pdf", width = 8, height = 6)  # You can adjust width and height as needed
netVisual_bubble(cellchat, sources.use = c("Tumor"), targets.use = c("Tumor","SPP1..Macro", "IL1B..Macro", "C1QC..Macro"), pairLR.use = pairLR.use, comparison = c(1, 2), angle.x = 45)
dev.off()

In [ ]:
pathways.show <- c("MIF") 
weight.max <- getMaxWeight(object.list, slot.name = c("netP"), attribute = pathways.show) # control the edge weights across different datasets
par(mfrow = c(1,2), xpd=TRUE)
for (i in 1:length(object.list)) {
  netVisual_aggregate(object.list[[i]], signaling = pathways.show, layout = "circle", edge.weight.max = weight.max[1], edge.width.max = 10, signaling.name = paste(pathways.show, names(object.list)[i]))
}

In [ ]:
# Spatial plot
pathways.show <- c("SPP1") 

netVisual_aggregate(cellchat_Niche4, signaling = pathways.show, sample.use = "NPC_ST16", layout = "spatial", 
sources.use = c("Tumor","SPP1..Macro", "IL1B..Macro", "C1QC..Macro"), 
targets.use = c("Tumor","SPP1..Macro", "IL1B..Macro", "C1QC..Macro"),
edge.width.max = 2, alpha.image = 0.2, vertex.size.max = 0.1, vertex.label.cex = 0, vertex.weight = 1)

netVisual_aggregate(cellchat_Niche4, signaling = pathways.show, sample.use = "NPC_ST17", layout = "spatial", 
sources.use = c("Tumor","SPP1..Macro", "IL1B..Macro", "C1QC..Macro"), 
targets.use = c("Tumor","SPP1..Macro", "IL1B..Macro", "C1QC..Macro"),
edge.width.max = 2, alpha.image = 0.2, vertex.size.max = 0.1, vertex.label.cex = 0, vertex.weight = 1)

netVisual_aggregate(cellchat_Niche4, signaling = pathways.show, sample.use = "NPC_ST18", layout = "spatial", 
sources.use = c("Tumor","SPP1..Macro", "IL1B..Macro", "C1QC..Macro"), 
targets.use = c("Tumor","SPP1..Macro", "IL1B..Macro", "C1QC..Macro"),
edge.width.max = 2, alpha.image = 0.2, vertex.size.max = 0.1, vertex.label.cex = 0, vertex.weight = 1)

netVisual_aggregate(cellchat_Niche4, signaling = pathways.show, sample.use = "NPC_ST19", layout = "spatial", 
sources.use = c("Tumor","SPP1..Macro", "IL1B..Macro", "C1QC..Macro"),
targets.use = c("Tumor","SPP1..Macro", "IL1B..Macro", "C1QC..Macro"),
edge.width.max = 2, alpha.image = 0.2, vertex.size.max = 0.1, vertex.label.cex = 0, vertex.weight = 1)

In [ ]:
library(ggplot2)
library(ggpubr)

plot1 <- netVisual_aggregate(cellchat_Niche4, signaling = c("MIF"), sample.use = "NPC_ST16", layout = "spatial", 
                             sources.use = c("Tumor", "SPP1..Macro", "IL1B..Macro", "C1QC..Macro"), 
                             targets.use = c("Tumor", "SPP1..Macro", "IL1B..Macro", "C1QC..Macro"),
                             edge.width.max = 2, alpha.image = 0.2, vertex.size.max = 0.1, vertex.label.cex = 0, vertex.weight = 1)

plot2 <- netVisual_aggregate(cellchat_Niche4, signaling = c("MIF"), sample.use = "NPC_ST17", layout = "spatial", 
                             sources.use = c("Tumor", "SPP1..Macro", "IL1B..Macro", "C1QC..Macro"), 
                             targets.use = c("Tumor", "SPP1..Macro", "IL1B..Macro", "C1QC..Macro"),
                             edge.width.max = 2, alpha.image = 0.2, vertex.size.max = 0.1, vertex.label.cex = 0, vertex.weight = 1)

plot3 <- netVisual_aggregate(cellchat_Niche4, signaling = c("MIF"), sample.use = "NPC_ST18", layout = "spatial", 
                             sources.use = c("Tumor", "SPP1..Macro", "IL1B..Macro", "C1QC..Macro"), 
                             targets.use = c("Tumor", "SPP1..Macro", "IL1B..Macro", "C1QC..Macro"),
                             edge.width.max = 2, alpha.image = 0.2, vertex.size.max = 0.1, vertex.label.cex = 0, vertex.weight = 1)

plot4 <- netVisual_aggregate(cellchat_Niche4, signaling = c("MIF"), sample.use = "NPC_ST19", layout = "spatial", 
                             sources.use = c("Tumor", "SPP1..Macro", "IL1B..Macro", "C1QC..Macro"),
                             targets.use = c("Tumor", "SPP1..Macro", "IL1B..Macro", "C1QC..Macro"),
                             edge.width.max = 2, alpha.image = 0.2, vertex.size.max = 0.1, vertex.label.cex = 0, vertex.weight = 1)

plot5 <- netVisual_aggregate(cellchat_Niche4, signaling = c("SPP1"), sample.use = "NPC_ST16", layout = "spatial", 
                             sources.use = c("Tumor", "SPP1..Macro", "IL1B..Macro", "C1QC..Macro"), 
                             targets.use = c("Tumor", "SPP1..Macro", "IL1B..Macro", "C1QC..Macro"),
                             edge.width.max = 2, alpha.image = 0.2, vertex.size.max = 0.1, vertex.label.cex = 0, vertex.weight = 1)

plot6 <- netVisual_aggregate(cellchat_Niche4, signaling = c("SPP1"), sample.use = "NPC_ST17", layout = "spatial", 
                             sources.use = c("Tumor", "SPP1..Macro", "IL1B..Macro", "C1QC..Macro"), 
                             targets.use = c("Tumor", "SPP1..Macro", "IL1B..Macro", "C1QC..Macro"),
                             edge.width.max = 2, alpha.image = 0.2, vertex.size.max = 0.1, vertex.label.cex = 0, vertex.weight = 1)

plot7 <- netVisual_aggregate(cellchat_Niche4, signaling = c("SPP1"), sample.use = "NPC_ST18", layout = "spatial", 
                             sources.use = c("Tumor", "SPP1..Macro", "IL1B..Macro", "C1QC..Macro"), 
                             targets.use = c("Tumor", "SPP1..Macro", "IL1B..Macro", "C1QC..Macro"),
                             edge.width.max = 2, alpha.image = 0.2, vertex.size.max = 0.1, vertex.label.cex = 0, vertex.weight = 1)

plot8 <- netVisual_aggregate(cellchat_Niche4, signaling = c("SPP1"), sample.use = "NPC_ST19", layout = "spatial", 
                             sources.use = c("Tumor", "SPP1..Macro", "IL1B..Macro", "C1QC..Macro"),
                             targets.use = c("Tumor", "SPP1..Macro", "IL1B..Macro", "C1QC..Macro"),
                             edge.width.max = 2, alpha.image = 0.2, vertex.size.max = 0.1, vertex.label.cex = 0, vertex.weight = 1)

combined_plot <- ggarrange(plot1, plot2, plot3, plot4, plot5, plot6, plot7, plot8,
                           ncol = 4, nrow = 2, 
                           common.legend = TRUE, legend = "top")

ggsave("Results/10.NPC_ST_Analysis/Spatial_cellchat_SPP1_MIF.pdf", combined_plot, width = 16, height = 8)

In [ ]:
sessionInfo()